In [40]:
import pickle
import random
import tensorflow as tf
import numpy as np

In [41]:
with open("feature_set_deepLearning","rb") as f:
    feature_set = pickle.load(f)
    
random.shuffle( feature_set )

In [42]:
print(len(feature_set[0][0]))

2259


In [43]:
n_nodes_hl1 = 500
n_nodes_hl2 = 500
n_nodes_hl3 = 500
batch_size = 100
n_classes = 2

X = tf.placeholder("float", [None, 2259])
y = tf.placeholder("float")

In [44]:
X_1 = []
y_1 = []

for feature in feature_set:
    X_1.append(feature[0])
    y_1.append(feature[1])

In [45]:
test_size = .2

train_X = X_1[:-int(len(X_1)*test_size)]
test_X = X_1[-int(len(X_1)*test_size):]
train_y = y_1[:-int(len(y_1)*test_size)]
test_y = y_1[-int(len(y_1)*test_size):]

In [46]:
def neural_network_model( data ):
    hidden_layer_1 = { "weights": tf.Variable(tf.random_normal([2259,n_nodes_hl1])),
                       "biases" : tf.Variable(tf.random_normal([n_nodes_hl1]))   }
    
    hidden_layer_2 = { "weights": tf.Variable(tf.random_normal([n_nodes_hl1,n_nodes_hl2])),
                       "biases" : tf.Variable(tf.random_normal([n_nodes_hl2]))   }
    
    hidden_layer_3 = { "weights": tf.Variable(tf.random_normal([n_nodes_hl2,n_nodes_hl3])),
                       "biases" : tf.Variable(tf.random_normal([n_nodes_hl3]))   }
    
    output_layer = { "weights" : tf.Variable(tf.random_normal([n_nodes_hl3,n_classes])),
                     "biases"  : tf.Variable(tf.random_normal([n_classes]))  }
    
    hl1 = tf.matmul( data,hidden_layer_1["weights"] ) + hidden_layer_1["biases"]
    hl1 = tf.tanh(hl1)
    
    hl2 = tf.matmul( hl1,hidden_layer_2["weights"] ) + hidden_layer_2["biases"]
    hl2 = tf.tanh(hl2)
    
    hl3 = tf.matmul( hl2,hidden_layer_3["weights"] ) + hidden_layer_3["biases"]
    hl3 = tf.tanh(hl3)
    
    output = tf.matmul( hl3,output_layer["weights"] ) + output_layer["biases"]
    
    return output
    

In [49]:
def train_neural_network( X ):
    predictions = neural_network_model(X)
    cost = tf.nn.softmax_cross_entropy_with_logits( logits=predictions,labels=y )
    optimizer = tf.train.AdamOptimizer().minimize(cost)
    
    hm_epochs = 10
    with tf.Session() as sess:
        sess.run( tf.global_variables_initializer() )
        
        for epoch in range(hm_epochs):
            start = 0
            end = start + batch_size
            epoch_cost = 0
            
            while start < len(train_X):
                epoch_x = train_X[start:end]
                epoch_y = train_y[start:end]
                
                c,_ = sess.run( [cost,optimizer],feed_dict = {X:epoch_x,y:epoch_y} )
                epoch_cost += c
                
                start = end
                end = min( len(train_X), start + batch_size)
            print ("After epoch number ", epoch+1 , " the cost is ", epoch_cost)
            
        correct = tf.equal( tf.argmax(predictions), tf.argmax(y) )
        accuracy = tf.reduce_mean(tf.cast(correct, "float"))
        print ("Accuracy is ", accuracy.eval(feed_dict = {X:test_X,y:test_y})  )
        
train_neural_network(X)
                    
        

After epoch number  1  the cost is  [197.58635  404.88284   61.52147  179.78859  247.08765  192.10802
 146.15952  200.5713   158.2302   178.45749  277.59088  135.07452
 234.51987  218.39474  132.65988  273.29132   67.83119  229.96416
 255.44382  304.58884  231.68338  215.19081  130.83563   83.65696
 187.07373  289.05502  187.38448  119.76293  300.05768  176.96234
 174.08539  233.64752  212.10468  205.15613  188.0693   203.83423
 202.20367  202.09052  196.65044   72.489235 199.7216   224.56308
 181.99721  102.11233  218.35341  182.32085  117.96892  149.32675
 362.1993   150.35161  138.30109  159.75662  222.94849  135.27339
 399.33386  143.50691  171.57082  263.86298  127.67159  129.62704
 129.78905  124.949066 266.7333   145.04553   81.44773  231.22092
 151.18042  204.43066  103.24902  120.09384  196.54362  264.6756
 184.58176  161.88199  113.05541  181.2554   221.64214  176.0941
 164.49162  127.2061   140.27312  175.17397  142.59143  233.40515
 255.96164  164.05966  164.71613  342.9739

After epoch number  7  the cost is  [6.99641630e-02 3.66695595e+01 3.83087853e-03 3.48750167e-02
 1.15099223e-02 6.26946017e-02 1.54933815e+01 5.56777716e-01
 6.50696903e-02 4.49767150e-02 2.19620660e-01 2.95054410e-02
 3.37975123e-03 2.63238449e+01 2.77747154e-01 1.31568685e-02
 2.82993424e-03 2.13026673e-01 4.13004456e+01 1.44988790e-01
 7.43031874e-02 1.14640560e+01 3.07282305e+00 3.49194519e-02
 1.01271011e-01 3.51206894e+01 1.94489326e+01 1.16259851e+01
 2.37374585e-02 6.09668851e-01 1.86130346e-03 8.39824323e-03
 1.25614290e+01 1.00690699e+01 1.16953328e-02 3.62909213e-03
 2.09328737e-02 1.41048878e-01 5.67670912e-03 5.20050526e-03
 4.49127816e-02 1.86490733e-02 2.55936896e-03 3.87244788e-03
 1.48212880e-01 4.92346880e-04 3.26845139e-01 1.90857029e+00
 3.59272308e+01 5.10880258e-03 4.10238057e-01 1.51952566e-03
 1.80628719e+01 1.82146745e-04 2.23849888e+01 1.83619820e-02
 1.16612778e+01 1.80333734e-01 8.62731226e-03 3.23957428e-02
 2.39875633e-02 2.74484253e+00 6.10093498e+00 2.9